In [2]:
import os
import cv2
import numpy as np
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template_string, request, redirect, Response, url_for, session
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from werkzeug.utils import secure_filename
from flask import Flask, render_template, request

app = Flask(__name__)
app.secret_key = 'your_secret_key'

# Load deepfake detection model
model = load_model("deepfake_model.h5")

# Configure upload settings
UPLOAD_FOLDER = "static/uploads"
ALLOWED_EXTENSIONS = {"png", "avif", "jpg", "jpeg", "mp4"}
app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER

def allowed_file(filename):
    return "." in filename and filename.rsplit(".", 1)[1].lower() in ALLOWED_EXTENSIONS

def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)[0][0]
    return "Fake" if prediction > 0.5 else "Real"

'''def predict_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count, fake_count = 0, 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        img = cv2.resize(frame, (224, 224))
        img = img_to_array(img) / 255.0
        img = np.expand_dims(img, axis=0)
        prediction = model.predict(img)[0][0]
        if prediction > 0.5:
            fake_count += 1
    cap.release()
    if frame_count == 0: return "0.00% of the video is fake"
    percentage_fake = (fake_count / frame_count) * 100
    return f"{percentage_fake:.2f}% of the video is fake" '''

def predict_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count, fake_count = 0, 0

    skip =3   # ← Skip every 3 frames
    current = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Skip frames -----------------------------------
        current += 1
        if current % skip != 0:
            continue
        # -------------------------------------------------

        frame_count += 1
        img = cv2.resize(frame, (224, 224))
        img = img_to_array(img) / 255.0
        img = np.expand_dims(img, axis=0)

        prediction = model.predict(img)[0][0]
        if prediction > 0.5:
            fake_count += 1

    cap.release()

    if frame_count == 0:
        return "0.00% of the video is fake"

    percentage_fake = (fake_count / frame_count) * 100
    return f"{percentage_fake:.2f}% of the video is fake"



def detect_live():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        img = cv2.resize(frame, (224, 224))
        img = img_to_array(img) / 255.0
        img = np.expand_dims(img, axis=0)
        prediction = model.predict(img)[0][0]
        label = "Fake" if prediction > 0.5 else "Real"
        color = (0, 0, 255) if label == "Fake" else (0, 255, 0)
        cv2.putText(frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        _, buffer = cv2.imencode(".jpg", frame)
        frame = buffer.tobytes()
        yield (b"--frame\r\nContent-Type: image/jpeg\r\n\r\n" + frame + b"\r\n")
    cap.release()

# ---------------- Login Page ----------------
@app.route("/", methods=["GET", "POST"])
def login():
    if request.method == "POST":
        username = request.form["username"]
        password = request.form["password"]

        if username == "admin" and password == "admin":
            session['user'] = username
            return redirect(url_for("home"))
        else:
            return render_template("login.html", error="Invalid credentials")

    return render_template("login.html")


# ---------------- Dashboard ----------------
@app.route("/home")
def home():
    if 'user' not in session:
        return redirect(url_for("login"))
    return render_template("home.html")


# ---------------- Image Result ----------------
@app.route("/upload-image", methods=["POST"])
def upload_image():
    if "file" not in request.files:
        return redirect(url_for("home"))

    file = request.files["file"]

    if file.filename == "" or not allowed_file(file.filename):
        return redirect(url_for("home"))

    filename = secure_filename(file.filename)
    filepath = os.path.join(app.config["UPLOAD_FOLDER"], filename)
    file.save(filepath)

    result = predict_image(filepath)
    icon = "fa-times-circle text-danger" if result == "Fake" else "fa-check-circle text-success"

    return render_template("image_result.html", result=result, icon=icon, filepath=filepath)


# ---------------- Video Result ----------------
@app.route("/upload-video", methods=["POST"])
def upload_video():
    if "file" not in request.files:
        return redirect(url_for("home"))

    file = request.files["file"]

    if file.filename == "" or not allowed_file(file.filename):
        return redirect(url_for("home"))

    filename = secure_filename(file.filename)
    filepath = os.path.join(app.config["UPLOAD_FOLDER"], filename)
    file.save(filepath)

    result = predict_video(filepath)

    percentage = float(result.split("%")[0])
    progress_color = "danger" if percentage > 50 else "warning" if percentage > 20 else "success"

    return render_template(
        "video_result.html",
        result=result,
        percentage=percentage,
        progress_color=progress_color
    )


# ---------------- Live Video ----------------
@app.route("/live-video")
def live_video():
    return Response(detect_live(), mimetype="multipart/x-mixed-replace; boundary=frame")

# ---------------- Logout ----------------
@app.route("/logout")
def logout():
    session.pop('user', None)
    return redirect(url_for("login"))

# ---------------- Main ----------------
if __name__ == "__main__":
    if not os.path.exists(UPLOAD_FOLDER):
        os.makedirs(UPLOAD_FOLDER)
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Nov/2025 10:19:35] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2025 10:19:36] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2025 10:19:44] "POST / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2025 10:19:44] "GET /home HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


INFO:werkzeug:127.0.0.1 - - [28/Nov/2025 10:20:10] "POST /upload-image HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2025 10:20:10] "GET /static/uploads/WhatsApp_Image_2025-11-28_at_10.10.41_AM.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2025 10:20:18] "GET /home HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step


...

INFO:werkzeug:127.0.0.1 - - [28/Nov/2025 10:23:25] "POST /upload-image HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2025 10:23:25] "GET /static/uploads/WhatsApp_Image_2025-11-28_at_10.10.41_AM.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2025 10:23:31] "GET /home HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step


INFO:werkzeug:127.0.0.1 - - [28/Nov/2025 10:25:01] "POST /upload-image HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2025 10:25:01] "GET /static/uploads/Gemini_Generated_Image_9dyvrc9dyvrc9dyv.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2025 10:25:06] "GET /home HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step


INFO:werkzeug:127.0.0.1 - - [28/Nov/2025 10:25:17] "POST /upload-image HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2025 10:25:17] "GET /static/uploads/WhatsApp_Image_2025-11-28_at_10.10.41_AM.jpeg HTTP/1.1" 200 -
